# Hypothesis testing

In [1]:
# Import packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, json, time
from tqdm.notebook import tqdm_notebook
import glob
import tmdbsimple as tmdb

In [2]:
with open('/Users/nicholasgiuffrida/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['client-id', 'api-key'])

In [3]:
#  Set the API_KEY
tmdb.API_KEY =  login['api-key']

In [4]:
# Create the folder for saving files (if it doesn't exist)
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
# Show the list of files included in the folder
sorted(os.listdir(FOLDER))

['.csv',
 '.ipynb_checkpoints',
 'B_filtered.csv',
 'Datatmdb_api_results_2001.json',
 'Datatmdb_api_results_2002.json',
 'Datatmdb_api_results_2003.json',
 'Datatmdb_api_results_2004.json',
 'Datatmdb_api_results_2005.json',
 'Datatmdb_api_results_2006.json',
 'Datatmdb_api_results_2007.json',
 'IMDB Movie Dataset Info.docx',
 'basics',
 'final_tmdb_data_2001.csv.gz',
 'final_tmdb_data_2002.csv.gz',
 'movies db erd.ndm2',
 'movies.db',
 'movies_db.png',
 'movies_db.sql',
 'ratings_filtered.csv',
 'title-akas-us-only.csv',
 'title.basics.tsv.gz',
 'title.ratings.tsv.gz',
 'tmdb_api_results_2001.json',
 'tmdb_api_results_2002.json']

In [5]:
def get_movie_with_rating(movie_id):
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
        # save the .info .releases dictionaries
    movie_info = movie.info()
    releases = movie.releases()
        # Loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation==US
        if c['iso_3166_1' ] =='US':
            ## save a "certification" key in the info dict with the certification
            movie_info['certification'] = c['certification']
    return movie_info

In [6]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        #  Load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Set file current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [7]:
YEARS_TO_GET = [2003,2004,2005,2006,2007]
# Create an empty list for saving errors
errors = []

In [8]:
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
  # create JSON file to store results  
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
   # Check if exists 
    file_exists = os.path.isfile(JSON_FILE)
    # If it does not exist: create it
    if file_exists == False:
        print(f"Creating {JSON_FILE} for API results for year={YEAR}.")
    # save empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
        # If it exists, print message
    else:
        print(f'The file {JSON_FILE} already exists.')

YEARS:   0%|          | 0/5 [00:00<?, ?it/s]

Creating Data/tmdb_api_results_2003.json for API results for year=2003.
Creating Data/tmdb_api_results_2004.json for API results for year=2004.
Creating Data/tmdb_api_results_2005.json for API results for year=2005.
Creating Data/tmdb_api_results_2006.json for API results for year=2006.
Creating Data/tmdb_api_results_2007.json for API results for year=2007.


In [9]:
basics = pd.read_csv('Data/B_filtered.csv')
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama
...,...,...,...,...,...,...,...,...,...
86974,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019.0,NaN,74,Drama
86975,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019.0,NaN,97,"Comedy,Drama,Fantasy"
86976,tt9916170,movie,The Rehearsal,O Ensaio,0,2019.0,NaN,51,Drama
86977,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller"


In [10]:
# Filter for movies from selected startYear
df = basics.loc[ basics['startYear']==YEAR].copy()
# Save movie ids to list
movie_ids = df['tconst']
movie_ids.head()

25      tt0119231
781     tt0211933
830     tt0213985
928     tt0218140
1090    tt0227671
Name: tconst, dtype: object

In [13]:
get_movie_with_rating('tt0213985')

{'adult': False,
 'backdrop_path': '/d9WT4FCjvB3Z6Sx2cYckdPi9rMI.jpg',
 'belongs_to_collection': None,
 'budget': 0,
 'genres': [{'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}],
 'homepage': '',
 'id': 14608,
 'imdb_id': 'tt0213985',
 'original_language': 'en',
 'original_title': 'Partition',
 'overview': 'Determined to leave the ravages of war behind, 38 year old Gian Singh resigns from the British Indian Army to a quiet life. His world is soon thrown in turmoil, when he suddenly finds himself responsible for the life of a 17 year old girl, traumatized by the events that separated her from her family.',
 'popularity': 5.675,
 'poster_path': '/qCq3QZbrf2ENAEyPJVaCGbWOQ1N.jpg',
 'production_companies': [{'id': 3905,
   'logo_path': None,
   'name': 'Khussro Films',
   'origin_country': ''}],
 'production_countries': [{'iso_3166_1': 'CA', 'name': 'Canada'},
  {'iso_3166_1': 'ZA', 'name': 'South Africa'},
  {'iso_3166_1': 'GB', 'name': 'United Kingdom'}],
 'release_date': '

In [11]:
# Load data from json into a dataframe
previous_df = pd.read_json(JSON_FILE)
previous_df

,imdb_id
0,0


In [12]:
# filter any ids that are already in the JSON_FILE
movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

In [13]:
# Loop through movie_ids_to_get with a tqdm progress bar
for movie_id in tqdm_notebook(movie_ids_to_get, f"Movies from {YEAR}"):
# Attempt to retrieve  data for the movie id
    try:
        temp = get_movie_with_rating(movie_id) 
        # Append/extend results to existing file using  pre-made function
        write_json(temp,JSON_FILE)
        # Short 20 ms sleep to prevent overwhelming server
        time.sleep(0.02)
        # If fails,  make a dict with just the id and None for certification.
    except Exception as e:
        errors.append([movie_id, e])

Movies from 2007:   0%|          | 0/2579 [00:00<?, ?it/s]

In [14]:
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
  # create JSON file to store results  
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
   # Check if exists 
    file_exists = os.path.isfile(JSON_FILE)
    # If it does not exist: create it
    if file_exists == False:
        print(f"Creating {JSON_FILE} for API results for year={YEAR}.")
    # save empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
        # If it exists, print message
    else:
        print(f'The file {JSON_FILE} already exists.')

YEARS:   0%|          | 0/5 [00:00<?, ?it/s]

The file Data/tmdb_api_results_2003.json already exists.
The file Data/tmdb_api_results_2004.json already exists.
The file Data/tmdb_api_results_2005.json already exists.
The file Data/tmdb_api_results_2006.json already exists.
The file Data/tmdb_api_results_2007.json already exists.


In [15]:
# Save the final results to a csv.gz file
final_year_df = pd.read_json(JSON_FILE)

csv_fname = f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz"
final_year_df.to_csv(csv_fname, compression="gzip", index=False)